In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [78]:
n = 10000
theta = np.linspace(0,np.pi*2, n)
# theta = np.linspace(0,np.pi*2, n)
r = 42_164 * 1000
earthR = 6_371 * 1000
sunR = 696_340 * 1000

x = r * np.cos(theta)
y = r * np.sin(theta)
z = np.zeros(n)


rSunX = 147_100_000_000
rSunY = 0
rSunZ = 0


In [82]:
plt.plot(theta,x)
plt.plot(theta,y)

In [3]:
def dot(x1,y1,z1,x2,y2,z2):
    return x1*x2 + y1*y2 + z1*z2

In [4]:
def s_c(x,y,z, rSunX, rSunY,rSunZ, planetR):
    r_rSun = dot(x,y,z, rSunX,rSunY,rSunZ)

    rSunMag = np.sqrt(float(rSunX * rSunX + rSunY * rSunY + rSunZ*rSunZ))

    r_rSun /=rSunMag

    rMag = np.sqrt(float(x*x + y*y + z*z))

    return r_rSun + np.sqrt(rMag*rMag - planetR*planetR)


In [5]:
def alpha_beta(x,y,z, rSunX, rSunY,rSunZ, rSun, rEarth): 
    dx = x - rSunX
    dy = y - rSunY
    dz = z - rSunZ

    mag = np.sqrt(float(dx*dx+ dy*dy + dz*dz))

    alpha = np.arctan((rSun - rEarth)/ mag)
    beta = np.arctan((rSun + rEarth)/ mag)

    return alpha,beta


In [6]:
def s_u(x,y,z, rSunX, rSunY,rSunZ, earthR, alpha):
    r_rSun = dot(x,y,z, rSunX,rSunY,rSunZ)

    rSunMag = np.sqrt(float(rSunX * rSunX + rSunY * rSunY + rSunZ*rSunZ))

    r_rSun /=rSunMag

    rMag = np.sqrt(float(x*x + y*y + z*z))

    return r_rSun + np.cos(alpha) * (np.sqrt(float(rMag * rMag - earthR * earthR * (np.cos(alpha)**2))) + earthR * np.sin(alpha))

def s_p(x,y,z, rSunX, rSunY,rSunZ, earthR, beta):
    r_rSun = dot(x,y,z, rSunX,rSunY,rSunZ)

    rSunMag = np.sqrt(float(rSunX * rSunX + rSunY * rSunY + rSunZ*rSunZ))

    r_rSun /=rSunMag

    rMag = np.sqrt(float(x*x + y*y + z*z))

    return r_rSun + np.cos(beta) * (np.sqrt(float(rMag * rMag - earthR * earthR * (np.cos(beta)**2))) - earthR * np.sin(beta))

In [12]:
def v_p(x,y,z,rSunX, rSunY, rSunZ, earthR, sunR):

    s_c_value = s_c(x,y,z,rSunX,rSunY,rSunZ, earthR)

    alpha,beta = alpha_beta(x,y,z, rSunX, rSunY, rSunZ, sunR, earthR)

    s_u_value = s_u(x,y,z,rSunX,rSunY,rSunZ, earthR, alpha)
    s_p_value = s_p(x,y,z,rSunX,rSunY,rSunZ, earthR, beta)

    delta_h = s_u_value - s_p_value
    
    small_delta = 8 

    return 0.5 * (1 + np.tanh( ( (small_delta * 2 * np.pi * earthR)/(delta_h) ) * s_c_value))

In [61]:
v_p_array = []
alpha_array = []
beta_array = []
s_c_array = []
s_u_array = []
s_p_array = []
for i in range(n):
    v_p_value = v_p(x[i], y[i], z[i], rSunX, rSunY, rSunZ, earthR, sunR)
    v_p_array.append(v_p_value)

    alpha_value,beta_value = alpha_beta(x[i],y[i],z[i],rSunX, rSunY, rSunZ,sunR,earthR)
    alpha_array.append(alpha_value)
    beta_array.append(beta_value)

    s_c_array.append(s_c(x[i],y[i],z[i],rSunX,rSunY, rSunZ, earthR))
    s_u_array.append(s_u(x[i],y[i],z[i],rSunX,rSunY, rSunZ, earthR, alpha_value))
    s_p_array.append(s_p(x[i],y[i],z[i],rSunX,rSunY, rSunZ, earthR, beta_value))

below_zero = False

for i in range(n):
    if not below_zero and s_p_array[i] < 0:
        in_zero = i
        below_zero = True

    if below_zero and s_p_array[i] > 0:
        out_zero = i
        below_zero = False


In [79]:
%matplotlib qt
plt.plot(theta[in_zero:out_zero], s_c_array[in_zero:out_zero], label="s_c")
plt.plot(theta[in_zero:out_zero], s_u_array[in_zero:out_zero], label="s_u")
plt.plot(theta[in_zero:out_zero], s_p_array[in_zero:out_zero], label="s_p")
# plt.plot(theta, s_c_array, label="s_c")
# plt.plot(theta, s_u_array, label="s_u")
# plt.plot(theta, s_p_array, label="s_p")
# plt.ylim(-2 * 10**-3, 2 * 10**-3)
plt.legend()
# plt.axvline(0, linestyle="dotted")
# plt.axvline(np.pi/2, linestyle="dotted")
# plt.axvline(np.pi, linestyle="dotted")
# plt.axvline(np.pi * 3/2, linestyle="dotted")
# plt.axvline(np.pi * 2, linestyle="dotted")

In [69]:
plt.plot(theta, s_c_array)
plt.axhline(y=0)

In [75]:
theta[in_zero]

2.9854398834293647

In [74]:
theta[out_zero]

3.298373805119077

In [76]:
x[in_zero]

-41650986.79902484

In [77]:
x[out_zero]

-41646858.09038133